# Activity: Build a random forest model

## **Introduction**


As you're learning, random forests are popular statistical learning algorithms. Some of their primary benefits include reducing variance, bias, and the chance of overfitting.

This activity is a continuation of the project you began modeling with decision trees for an airline. Here, you will train, tune, and evaluate a random forest model using data from spreadsheet of survey responses from 129,880 customers. It includes data points such as class, flight distance, and inflight entertainment. Your random forest model will be used to predict whether a customer will be satisfied with their flight experience.

**Note:** Because this lab uses a real dataset, this notebook first requires exploratory data analysis, data cleaning, and other manipulations to prepare it for modeling.

## **Step 1: Imports** 


Import relevant Python libraries and modules, including `numpy` and `pandas`libraries for data processing; the `pickle` package to save the model; and the `sklearn` library, containing:
- The module `ensemble`, which has the function `RandomForestClassifier`
- The module `model_selection`, which has the functions `train_test_split`, `PredefinedSplit`, and `GridSearchCV` 
- The module `metrics`, which has the functions `f1_score`, `precision_score`, `recall_score`, and `accuracy_score`


In [53]:
# Import `numpy`, `pandas`, `pickle`, and `sklearn`.
# Import the relevant functions from `sklearn.ensemble`, `sklearn.model_selection`, and `sklearn.metrics`.

### YOUR CODE HERE ###

## Import
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split, PredefinedSplit
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

As shown in this cell, the dataset has been automatically loaded in for you. You do not need to download the .csv file, or provide more code, in order to access the dataset and proceed with this lab. Please continue with this activity by completing the following instructions.

In [15]:
# RUN THIS CELL TO IMPORT YOUR DATA. 

### YOUR CODE HERE ###

air_data = pd.read_csv("Invistico_Airline.csv")

<details>
  <summary><h4><strong>Hint 1</strong></h4></summary>

The `read_csv()` function from the `pandas` library can be helpful here.
 
</details>

Now, you're ready to begin cleaning your data. 

## **Step 2: Data cleaning** 

To get a sense of the data, display the first 10 rows.

In [16]:
# Display first 10 rows.

### YOUR CODE HERE ###
air_data.head(10)

,satisfaction,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,Inflight wifi service,Inflight entertainment,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes
0,satisfied,Loyal Customer,65,Personal Travel,Eco,265,0,0,0,2,2,4,2,3,3,0,3,5,3,2,0,0.0
1,satisfied,Loyal Customer,47,Personal Travel,Business,2464,0,0,0,3,0,2,2,3,4,4,4,2,3,2,310,305.0
2,satisfied,Loyal Customer,15,Personal Travel,Eco,2138,0,0,0,3,2,0,2,2,3,3,4,4,4,2,0,0.0
3,satisfied,Loyal Customer,60,Personal Travel,Eco,623,0,0,0,3,3,4,3,1,1,0,1,4,1,3,0,0.0
4,satisfied,Loyal Customer,70,Personal Travel,Eco,354,0,0,0,3,4,3,4,2,2,0,2,4,2,5,0,0.0
5,satisfied,Loyal Customer,30,Personal Travel,Eco,1894,0,0,0,3,2,0,2,2,5,4,5,5,4,2,0,0.0
6,satisfied,Loyal Customer,66,Personal Travel,Eco,227,0,0,0,3,2,5,5,5,5,0,5,5,5,3,17,15.0
7,satisfied,Loyal Customer,10,Personal Travel,Eco,1812,0,0,0,3,2,0,2,2,3,3,4,5,4,2,0,0.0
8,satisfied,Loyal Customer,56,Personal Travel,Business,73,0,0,0,3,5,3,5,4,4,0,1,5,4,4,0,0.0
9,satisfied,Loyal Customer,22,Personal Travel,Eco,1556,0,0,0,3,2,0,2,2,2,4,5,3,4,2,30,26.0


<details>
  <summary><h4><strong>Hint 1</strong></h4></summary>

The `head()` function from the `pandas` library can be helpful here.
 
</details>

Now, display the variable names and their data types. 

In [17]:
# Display variable names and types.

### YOUR CODE HERE ###
air_data.dtypes

satisfaction                          object
Customer Type                         object
Age                                    int64
Type of Travel                        object
Class                                 object
Flight Distance                        int64
Seat comfort                           int64
Departure/Arrival time convenient      int64
Food and drink                         int64
Gate location                          int64
Inflight wifi service                  int64
Inflight entertainment                 int64
Online support                         int64
Ease of Online booking                 int64
On-board service                       int64
Leg room service                       int64
Baggage handling                       int64
Checkin service                        int64
Cleanliness                            int64
Online boarding                        int64
Departure Delay in Minutes             int64
Arrival Delay in Minutes             float64
dtype: obj

<details>
  <summary><h4><strong>Hint 1</strong></h4></summary>

DataFrames have an attribute that outputs variable names and data types in one result.
 
</details>

**Question:** What do you observe about the differences in data types among the variables included in the data?
`objects = ['Satisfaction','Customer Type','Type of Travel', 'Class']`
The rest is numerical.
We have continious numerical, categorical, and ordinal information.
Some features (objects) require transformation. some binary some ordinal.


Next, to understand the size of the dataset, identify the number of rows and the number of columns.

In [18]:
# Identify the number of rows and the number of columns.

### YOUR CODE HERE ###
air_data.shape

(129880, 22)

<details>
  <summary><h4><strong>Hint 1</strong></h4></summary>

There is a method in the `pandas` library that outputs the number of rows and the number of columns in one result.

</details>

Now, check for missing values in the rows of the data. Start with .isna() to get Booleans indicating whether each value in the data is missing. Then, use .any(axis=1) to get Booleans indicating whether there are any missing values along the columns in each row. Finally, use .sum() to get the number of rows that contain missing values.

In [19]:
# Get Booleans to find missing values in data.
# Get Booleans to find missing values along columns.
# Get the number of rows that contain missing values.

### YOUR CODE HERE ###
def miss_df(dataframe: pd.DataFrame):

    """
    Take a pandas df as argument, returns another one
    with  basic information about missing data
    Args:
        df(pd.DataFrame): a pdDataFrame.
    """

    # Running validation on the argument recieved
    assert type(dataframe) == pd.DataFrame, f'{dataframe}, is not a pandas df.'
    df = dataframe.copy()
    total_missing = df.isnull().sum().sort_values(ascending=False)
    percent_missing = (df.isnull().sum() / df.isnull().count()) * 100
    missing_data = pd.concat([total_missing, percent_missing], axis=1, keys=['Total', 'Percent'])
    return missing_data.head(len(df.columns))

miss_df(air_data)

,Total,Percent
Arrival Delay in Minutes,393,0.302587
Customer Type,0,0.000000
Departure Delay in Minutes,0,0.000000
Online boarding,0,0.000000
Cleanliness,0,0.000000
Checkin service,0,0.000000
Baggage handling,0,0.000000
Leg room service,0,0.000000
On-board service,0,0.000000
Ease of Online booking,0,0.000000


**Question:** How many rows of data are missing values?**
```
393 rows, Arrival Delay in Minutes...dtype: int64
```

Drop the rows with missing values. This is an important step in data cleaning, as it makes the data more useful for analysis and regression. Then, save the resulting pandas DataFrame in a variable named `air_data_subset`.

In [7]:
# Drop missing values.
# Save the DataFrame in variable `air_data_subset`.

### YOUR CODE HERE ###
air_data.dropna(inplace=True)
air_data_subset = air_data.copy()


<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

The `dropna()` function is helpful here.
</details>

<details>
<summary><h4><strong>Hint 2</strong></h4></summary>

The axis parameter passed in to this function should be set to 0 (if you want to drop rows containing missing values) or 1 (if you want to drop columns containing missing values).
</details>

Next, display the first 10 rows to examine the data subset.

In [8]:
# Display the first 10 rows.

### YOUR CODE HERE ###
air_data_subset.head(10)

,satisfaction,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,Inflight wifi service,Inflight entertainment,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes
0,satisfied,Loyal Customer,65,Personal Travel,Eco,265,0,0,0,2,2,4,2,3,3,0,3,5,3,2,0,0.0
1,satisfied,Loyal Customer,47,Personal Travel,Business,2464,0,0,0,3,0,2,2,3,4,4,4,2,3,2,310,305.0
2,satisfied,Loyal Customer,15,Personal Travel,Eco,2138,0,0,0,3,2,0,2,2,3,3,4,4,4,2,0,0.0
3,satisfied,Loyal Customer,60,Personal Travel,Eco,623,0,0,0,3,3,4,3,1,1,0,1,4,1,3,0,0.0
4,satisfied,Loyal Customer,70,Personal Travel,Eco,354,0,0,0,3,4,3,4,2,2,0,2,4,2,5,0,0.0
5,satisfied,Loyal Customer,30,Personal Travel,Eco,1894,0,0,0,3,2,0,2,2,5,4,5,5,4,2,0,0.0
6,satisfied,Loyal Customer,66,Personal Travel,Eco,227,0,0,0,3,2,5,5,5,5,0,5,5,5,3,17,15.0
7,satisfied,Loyal Customer,10,Personal Travel,Eco,1812,0,0,0,3,2,0,2,2,3,3,4,5,4,2,0,0.0
8,satisfied,Loyal Customer,56,Personal Travel,Business,73,0,0,0,3,5,3,5,4,4,0,1,5,4,4,0,0.0
9,satisfied,Loyal Customer,22,Personal Travel,Eco,1556,0,0,0,3,2,0,2,2,2,4,5,3,4,2,30,26.0


Confirm that it does not contain any missing values.

In [21]:
# Count of missing values.

### YOUR CODE HERE ###
miss_df(air_data_subset)

,Total,Percent
satisfaction,0,0.0
Customer Type,0,0.0
Departure Delay in Minutes,0,0.0
Online boarding,0,0.0
Cleanliness,0,0.0
Checkin service,0,0.0
Baggage handling,0,0.0
Leg room service,0,0.0
On-board service,0,0.0
Ease of Online booking,0,0.0


<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

You can use the `.isna().sum()` to get the number of missing values for each variable.

</details>

Next, convert the categorical features to indicator (one-hot encoded) features. 

**Note:** The `drop_first` argument can be kept as default (`False`) during one-hot encoding for random forest models, so it does not need to be specified. Also, the target variable, `satisfaction`, does not need to be encoded and will be extracted in a later step.

In [33]:
# Convert categorical features to one-hot encoded features.

### YOUR CODE HERE ###
def str_encoder(dataframe, str_to_encode):
    """
    The str_encoder function will convert these columns 
    to numerical ordinal or binary numerical columns and 
    then return a new DataFrame with the transformed columns 
    plus the original columns that were not transformed.
    
    Arguments:
        df: A Pandas Data frame.
        str_to_encode: a list of columns to tranform 
        from the same data set.
    Returns:
        A new data set made of the transformed columns
        plus the original with out the previously transformed
        data.
    """
    ## make a copy 
    df_o = dataframe.copy()
    df_e = df_o[str_to_encode].copy()
    
    le = LabelEncoder()
    str_encoded = df_e.apply(le.fit_transform)

    ## Drop strings categories
    df_o_e = df_o.drop(str_to_encode, axis=1)

    ## Add transformed data
    full = df_o_e.join(str_encoded)
    
    return full

def observe_data_type(df: pd.DataFrame, max_class):

    """
    Args: 
        df(pd.DataFrame): a pdDataFrame.
        max_class: max number to be considered a categorical feature.
        Each case is different and EDA will tell.

    Returns:
        A new dataset with information related to unique values
        per column
    
    """
    # Running validation on the argument recieved
    assert type(df) == pd.DataFrame, f'{df}, is not a pandas df.'
    num_uniques = []
    column_name = []
    check = []
    resu = {}
    result = pd.DataFrame({})
    
    for c in df.columns:
        
        num_uniques.append(len(df[c].unique()))
        column_name.append(c)
        if 2 < len(df[c].unique()) < max_class:
            check.append('yes')
        else: check.append('no')
            
    
    result['columns_name'] = column_name
    result['Num_uniques'] = num_uniques
    result['possible_categorical'] = check
    return(result) 

def hot_encoder(dataframe, ordinal_to_hot_encode):
    
    """
    The hot_encoder function will convert the ordinal_to_hot_encode
    columns from numerical ordinal and then return a new DataFrame 
    with the transformed columns plus the original columns that were not transformed.
    
    Arguments:
        df: A Pandas Data frame.
        str_to_encode: a list of columns to tranform 
        from the same data set.
    Returns:
        A new data set made of the transformed columns
        plus the original with out the previously transformed
        data.
    """
    ## Original set
    original = dataframe.copy()
    
    ## Set minus the Transformed columns
    nums = original.drop(ordinal_to_hot_encode, axis=1)
    
    ## this transfomr to str
    str_encoded = original[ordinal_to_hot_encode].astype(str)

    ## thisgets the categories.
    categories = [original[i].unique().astype(str) for i in ordinal_to_hot_encode]
    
    ## update categories to remove duplicates
    categories = [list(set(cat)) for cat in categories]
    
    ## Using get_dummies
    encoded_set = pd.get_dummies(str_encoded, dtype=int)

    
    ## Using join with the nums data set and encoded_set
    full = nums.join(encoded_set)
    
    return full

## Selecting columns to transform
cat_str = ['satisfaction','Customer Type','Type of Travel', 'Class']

## Encode the categorical objects data type data
encoded_string = str_encoder(air_data_subset, cat_str)

## Selecting categorical features to transform
info = observe_data_type(encoded_string, 10)

to_hot_encode = info.loc[info['possible_categorical'] == 'yes', :]['columns_name'].tolist()
to_hot_encode

air_data_subset_dummies = hot_encoder(encoded_string, to_hot_encode)

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

You can use the `pd.get_dummies()` function to convert categorical variables to one-hot encoded variables.
</details>

**Question:** Why is it necessary to convert categorical data into dummy variables?**

Because models tend to have better preformance with data that has been brought to certain standards.
Bbjects needs to be brought to numerical.
When algorhitms are used to create models there is assumtions to be checked, but Random Forest is
not a deterministic model, is stochiast. Is a good practice.



Next, display the first 10 rows to review the `air_data_subset_dummies`. 

In [35]:
# Display the first 10 rows.

### YOUR CODE HERE ###
air_data_subset_dummies.head()

,Age,Flight Distance,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction,Customer Type,Type of Travel,Seat comfort_0,Seat comfort_1,Seat comfort_2,Seat comfort_3,Seat comfort_4,Seat comfort_5,Departure/Arrival time convenient_0,Departure/Arrival time convenient_1,Departure/Arrival time convenient_2,Departure/Arrival time convenient_3,Departure/Arrival time convenient_4,Departure/Arrival time convenient_5,Food and drink_0,Food and drink_1,Food and drink_2,Food and drink_3,Food and drink_4,Food and drink_5,Gate location_0,Gate location_1,Gate location_2,Gate location_3,Gate location_4,Gate location_5,Inflight wifi service_0,Inflight wifi service_1,Inflight wifi service_2,Inflight wifi service_3,Inflight wifi service_4,Inflight wifi service_5,Inflight entertainment_0,Inflight entertainment_1,Inflight entertainment_2,Inflight entertainment_3,Inflight entertainment_4,Inflight entertainment_5,Online support_0,Online support_1,Online support_2,Online support_3,Online support_4,Online support_5,Ease of Online booking_0,Ease of Online booking_1,Ease of Online booking_2,Ease of Online booking_3,Ease of Online booking_4,Ease of Online booking_5,On-board service_0,On-board service_1,On-board service_2,On-board service_3,On-board service_4,On-board service_5,Leg room service_0,Leg room service_1,Leg room service_2,Leg room service_3,Leg room service_4,Leg room service_5,Baggage handling_1,Baggage handling_2,Baggage handling_3,Baggage handling_4,Baggage handling_5,Checkin service_0,Checkin service_1,Checkin service_2,Checkin service_3,Checkin service_4,Checkin service_5,Cleanliness_0,Cleanliness_1,Cleanliness_2,Cleanliness_3,Cleanliness_4,Cleanliness_5,Online boarding_0,Online boarding_1,Online boarding_2,Online boarding_3,Online boarding_4,Online boarding_5,Class_0,Class_1,Class_2
0,65,265,0,0.0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
1,47,2464,310,305.0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0
2,15,2138,0,0.0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0
3,60,623,0,0.0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0
4,70,354,0,0.0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0


Then, check the variables of air_data_subset_dummies.

In [36]:
# Display variables.

### YOUR CODE HERE ###
air_data_subset_dummies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 129487 entries, 0 to 129879
Data columns (total 93 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   Age                                  129487 non-null  int64  
 1   Flight Distance                      129487 non-null  int64  
 2   Departure Delay in Minutes           129487 non-null  int64  
 3   Arrival Delay in Minutes             129487 non-null  float64
 4   satisfaction                         129487 non-null  int64  
 5   Customer Type                        129487 non-null  int64  
 6   Type of Travel                       129487 non-null  int64  
 7   Seat comfort_0                       129487 non-null  int64  
 8   Seat comfort_1                       129487 non-null  int64  
 9   Seat comfort_2                       129487 non-null  int64  
 10  Seat comfort_3                       129487 non-null  int64  
 11  Seat comfort_4    

**Question:** What changes do you observe after converting the string data to dummy variables?**

We have a higher number of columns.

## **Step 3: Model building** 

The first step to building your model is separating the labels (y) from the features (X).

In [46]:
# Separate the dataset into labels (y) and features (X).

### YOUR CODE HERE ###
y = air_data_subset_dummies['satisfaction']
X = air_data_subset_dummies[['Age',
 'Flight Distance',
 'Departure Delay in Minutes',
 'Arrival Delay in Minutes',
 'Customer Type',
 'Type of Travel',
 'Seat comfort_0',
 'Seat comfort_1',
 'Seat comfort_2',
 'Seat comfort_3',
 'Seat comfort_4',
 'Seat comfort_5',
 'Departure/Arrival time convenient_0',
 'Departure/Arrival time convenient_1',
 'Departure/Arrival time convenient_2',
 'Departure/Arrival time convenient_3',
 'Departure/Arrival time convenient_4',
 'Departure/Arrival time convenient_5',
 'Food and drink_0',
 'Food and drink_1',
 'Food and drink_2',
 'Food and drink_3',
 'Food and drink_4',
 'Food and drink_5',
 'Gate location_0',
 'Gate location_1',
 'Gate location_2',
 'Gate location_3',
 'Gate location_4',
 'Gate location_5',
 'Inflight wifi service_0',
 'Inflight wifi service_1',
 'Inflight wifi service_2',
 'Inflight wifi service_3',
 'Inflight wifi service_4',
 'Inflight wifi service_5',
 'Inflight entertainment_0',
 'Inflight entertainment_1',
 'Inflight entertainment_2',
 'Inflight entertainment_3',
 'Inflight entertainment_4',
 'Inflight entertainment_5',
 'Online support_0',
 'Online support_1',
 'Online support_2',
 'Online support_3',
 'Online support_4',
 'Online support_5',
 'Ease of Online booking_0',
 'Ease of Online booking_1',
 'Ease of Online booking_2',
 'Ease of Online booking_3',
 'Ease of Online booking_4',
 'Ease of Online booking_5',
 'On-board service_0',
 'On-board service_1',
 'On-board service_2',
 'On-board service_3',
 'On-board service_4',
 'On-board service_5',
 'Leg room service_0',
 'Leg room service_1',
 'Leg room service_2',
 'Leg room service_3',
 'Leg room service_4',
 'Leg room service_5',
 'Baggage handling_1',
 'Baggage handling_2',
 'Baggage handling_3',
 'Baggage handling_4',
 'Baggage handling_5',
 'Checkin service_0',
 'Checkin service_1',
 'Checkin service_2',
 'Checkin service_3',
 'Checkin service_4',
 'Checkin service_5',
 'Cleanliness_0',
 'Cleanliness_1',
 'Cleanliness_2',
 'Cleanliness_3',
 'Cleanliness_4',
 'Cleanliness_5',
 'Online boarding_0',
 'Online boarding_1',
 'Online boarding_2',
 'Online boarding_3',
 'Online boarding_4',
 'Online boarding_5',
 'Class_0',
 'Class_1',
 'Class_2']]

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Save the labels (the values in the `satisfaction` column) as `y`.

Save the features as `X`. 

</details>

<details>
<summary><h4><strong>Hint 2</strong></h4></summary>

To obtain the features, drop the `satisfaction` column from the DataFrame.

</details>

Once separated, split the data into train, validate, and test sets. 

In [47]:
# Separate into train, validate, test sets.

### YOUR CODE HERE ###
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size = 0.25, random_state = 42)

### Tune the model

Now, fit and tune a random forest model with separate validation set. Begin by determining a set of hyperparameters for tuning the model using GridSearchCV.


In [48]:
# Determine set of hyperparameters.

### YOUR CODE HERE ###
cv_params = {'n_estimators' : [50,100], 
              'max_depth' : [10,50],        
              'min_samples_leaf' : [0.5,1], 
              'min_samples_split' : [0.001, 0.01],
              'max_features' : ["sqrt"], 
              'max_samples' : [.5,.9]}

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Create a dictionary `cv_params` that maps each hyperparameter name to a list of values. The GridSearch you conduct will set the hyperparameter to each possible value, as specified, and determine which value is optimal.

</details>

<details>
<summary><h4><strong>Hint 2</strong></h4></summary>

The main hyperparameters here include `'n_estimators', 'max_depth', 'min_samples_leaf', 'min_samples_split', 'max_features', and 'max_samples'`. These will be the keys in the dictionary `cv_params`.

</details>

Next, create a list of split indices.

In [51]:
# Create list of split indices.

### YOUR CODE HERE ###
split_index = [0 if x in X_val.index else -1 for x in X_train.index]
custom_split = PredefinedSplit(split_index)

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Use list comprehension, iterating over the indices of `X_train`. The list can consists of 0s to indicate data points that should be treated as validation data and -1s to indicate data points that should be treated as training data.

</details>

<details>
<summary><h4><strong>Hint 2</strong></h4></summary>

Use `PredfinedSplit()`, passing in `split_index`, saving the output as `custom_split`. This will serve as a custom split that will identify which data points from the train set should be treated as validation data during GridSearch.

</details>

Now, instantiate your model.

In [54]:
# Instantiate model.

### YOUR CODE HERE ### 
rf = RandomForestClassifier(random_state=42)

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Use `RandomForestClassifier()`, specifying the `random_state` argument for reproducible results. This will help you instantiate a random forest model, `rf`.

</details>

Next, use GridSearchCV to search over the specified parameters.

In [55]:
# Search over specified parameters.

### YOUR CODE HERE ### 
rf_val = GridSearchCV(rf, cv_params, cv=custom_split, refit='f1', n_jobs = -1, verbose = 1)

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Use `GridSearchCV()`, passing in `rf` and `cv_params` and specifying `cv` as `custom_split`. Additional arguments that you can specify include: `refit='f1', n_jobs = -1, verbose = 1`. 

</details>

Now, fit your model.

In [59]:
%%timeit
# Fit the model.

### YOUR CODE HERE ###

rf_val.fit(X_train, y_train)

Fitting 1 folds for each of 32 candidates, totalling 32 fits
Fitting 1 folds for each of 32 candidates, totalling 32 fits
Fitting 1 folds for each of 32 candidates, totalling 32 fits
Fitting 1 folds for each of 32 candidates, totalling 32 fits
Fitting 1 folds for each of 32 candidates, totalling 32 fits
Fitting 1 folds for each of 32 candidates, totalling 32 fits
Fitting 1 folds for each of 32 candidates, totalling 32 fits
Fitting 1 folds for each of 32 candidates, totalling 32 fits
24.5 s ± 739 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Finally, obtain the optimal parameters.

In [61]:
# Obtain optimal parameters.

### YOUR CODE HERE ###
rf_val.best_params_

{'max_depth': 50,
 'max_features': 'sqrt',
 'max_samples': 0.9,
 'min_samples_leaf': 1,
 'min_samples_split': 0.001,
 'n_estimators': 100}

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Use the `best_params_` attribute to obtain the optimal values for the hyperparameters from the GridSearchCV model.

</details>

## **Step 4: Results and evaluation** 

Use the selected model to predict on your test data. Use the optimal parameters found via GridSearchCV.

In [62]:
# Use optimal parameters on GridSearchCV.

### YOUR CODE HERE ###
rf_opt = RandomForestClassifier(**rf_val.best_params_)

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Use `RandomForestClassifier()`, specifying the `random_state` argument for reproducible results and passing in the optimal hyperparameters found in the previous step. To distinguish this from the previous random forest model, consider naming this variable `rf_opt`.

</details>

Once again, fit the optimal model.

In [64]:
%%time
# Fit the optimal model.

### YOUR CODE HERE ###
rf_opt.fit(X_train, y_train)

CPU times: user 7.19 s, sys: 0 ns, total: 7.19 s
Wall time: 7.19 s


RandomForestClassifier(max_depth=50, max_samples=0.9, min_samples_split=0.001)

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Use the `fit()` method to train `rf_opt` on `X_train` and `y_train`.

</details>

And predict on the test set using the optimal model.

In [65]:
# Predict on test set.

### YOUR CODE HERE ###
y_pred = rf_opt.predict(X_test)

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

You can call the `predict()` function to make predictions on `X_test` using `rf_opt`. Save the predictions now (for example, as `y_pred`), to use them later for comparing to the true labels. 

</details>

### Obtain performance scores

First, get your precision score.

In [70]:
# Get precision score.

### YOUR CODE HERE ###
pc_test = precision_score(y_test, y_pred, pos_label=1)
print("The precision score is {pc:.3f}".format(pc = pc_test))

The precision score is 0.947


<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

You can call the `precision_score()` function from `sklearn.metrics`, passing in `y_test` and `y_pred` and specifying the `pos_label` argument as `"satisfied"`.
</details>

Then, collect the recall score.

In [71]:
# Get recall score.

### YOUR CODE HERE ###
rc_test = recall_score(y_test, y_pred, pos_label=1)
print("The recall score is {rc:.3f}".format(rc = rc_test))

The recall score is 0.939


<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

You can call the `recall_score()` function from `sklearn.metrics`, passing in `y_test` and `y_pred` and specifying the `pos_label` argument as `"satisfied"`.
</details>

Next, obtain your accuracy score.

In [72]:
# Get accuracy score.

### YOUR CODE HERE ###
ac_test = accuracy_score(y_test, y_pred)
print("The accuracy score is {ac:.3f}".format(ac = ac_test))

The accuracy score is 0.938


<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

You can call the `accuracy_score()` function from `sklearn.metrics`, passing in `y_test` and `y_pred` and specifying the `pos_label` argument as `"satisfied"`.
</details>

Finally, collect your F1-score.

In [74]:
# Get F1 score.

### YOUR CODE HERE ###
f1_test = f1_score(y_test, y_pred, pos_label=1)
print("The F1 score is {f1:.3f}".format(f1 = f1_test))

The F1 score is 0.943


<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

You can call the `f1_score()` function from `sklearn.metrics`, passing in `y_test` and `y_pred` and specifying the `pos_label` argument as `"satisfied"`.
</details>

**Question:** How is the F1-score calculated?

F1 = 2 * (precision * recall) / (precision + recall)

**Question:** What are the pros and cons of performing the model selection using test data instead of a separate validation dataset?

**Question:** What are the pros and cons of performing the model selection using test data instead of a separate validation dataset?

Pros:
*  The coding workload is reduced.
*  The scripts for data splitting are shorter.
*  It's only  necessary to evaluate test dataset performance once, instead of two evaluations (validate and test).

Cons:
* If a model is evaluated using samples that were also used to build or fine-tune that model, it likely will provide a biased evaluation.
* A potential overfitting issue could happen when fitting the model's scores on the test data.


### Evaluate the model

Now that you have results, evaluate the model. 

**Question:** What are the four basic parameters for evaluating the performance of a classification model?

1. True positives (TP): These are correctly predicted positive values, which means the value of actual and predicted classes are positive. 

2. True negatives (TN): These are correctly predicted negative values, which means the value of the actual and predicted classes are negative.

3. False positives (FP): This occurs when the value of the actual class is negative and the value of the predicted class is positive.

4. False negatives (FN): This occurs when the value of the actual class is positive and the value of the predicted class in negative. 

**Reminder:** When fitting and tuning classification modeld, data professioals aim to minimize false positives and false negatives.

**Question:**  What do the four scores demonstrate about your model, and how do you calculate them?

- Accuracy (TP+TN/TP+FP+FN+TN): The ratio of correctly predicted observations to total observations. 
 
- Precision (TP/TP+FP): The ratio of correctly predicted positive observations to total predicted positive observations. 

- Recall (Sensitivity, TP/TP+FN): The ratio of correctly predicted positive observations to all observations in actual class.

- F1 score: The harmonic average of precision and recall, which takes into account both false positives and false negatives. 

Calculate the scores: precision score, recall score, accuracy score, F1 score.

In [75]:
# Precision score on test data set.

### YOUR CODE HERE ###
# Precision score on test data set.

### YOUR CODE HERE ###

print("\nThe precision score is: {pc:.3f}".format(pc = pc_test), "for the test set,", "\nwhich means of all positive predictions,", "{pc_pct:.1f}% prediction are true positive.".format(pc_pct = pc_test * 100))


The precision score is: 0.947 for the test set, 
which means of all positive predictions, 94.7% prediction are true positive.


In [76]:
# Recall score on test data set.

### YOUR CODE HERE ###
# Recall score on test data set.

### YOUR CODE HERE ###

print("\nThe recall score is: {rc:.3f}".format(rc = rc_test), "for the test set,", "\nwhich means of which means of all real positive cases in test set,", "{rc_pct:.1f}% are  predicted positive.".format(rc_pct = rc_test * 100))


The recall score is: 0.939 for the test set, 
which means of which means of all real positive cases in test set, 93.9% are  predicted positive.


In [77]:
# Accuracy score on test data set.

### YOUR CODE HERE ###

print("\nThe accuracy score is: {ac:.3f}".format(ac = ac_test), "for the test set,", "\nwhich means of all cases in test set,", "{ac_pct:.1f}% are predicted true positive or true negative.".format(ac_pct = ac_test * 100))


The accuracy score is: 0.938 for the test set, 
which means of all cases in test set, 93.8% are predicted true positive or true negative.


In [78]:
# F1 score on test data set.

### YOUR CODE HERE ###

print("\nThe F1 score is: {f1:.3f}".format(f1 = f1_test), "for the test set,", "\nwhich means the test set's harmonic mean is {f1_pct:.1f}%.".format(f1_pct = f1_test * 100))


The F1 score is: 0.943 for the test set, 
which means the test set's harmonic mean is 94.3%.


**Question:** How does this model perform based on the four scores?

[Write your response here. Double-click (or enter) to edit.]

### Evaluate the model

Finally, create a table of results that you can use to evaluate the performace of your model.

In [87]:
# Create table of results.

### YOUR CODE HERE ###
table = {}
table['Model'] = 'Tuned Decision Tree'
table['F1'] = f1_score(y_test, y_pred)
table['Recall'] = recall_score(y_test, y_pred)
table['Precision'] = precision_score(y_test, y_pred)
table['Accuracy'] = accuracy_score(y_test, y_pred)

tabla = pd.DataFrame(table, index=['']).T
tabla

,
Model,Tuned Decision Tree
F1,0.943382
Recall,0.939353
Precision,0.947447
Accuracy,0.938434



<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Build a table to compare the performance of the models. Create a DataFrame and use the `append()` function to add the results of each model as a new row.

</details>

**Question:** How does the random forest model compare to the decision tree model you built in the previous lab?

The tuned random forest has higher scores overall, so it is the better model. Particularly, it shows a better F1 score than the decision tree model, which indicates that the random forest model may do better at classification when taking into account false positives and false negatives. 



## **Considerations**


**What are the key takeaways from this lab?**
- Data exploring, cleaning, and encoding are necessary for model building.
- A separate validation set is typically used for tuning a model, rather than using the test set. This also helps avoid the evaluation becoming biased.
-  F1 scores are usually more useful than accuracy scores. If the cost of false positives and false negatives are very different, it’s better to use the F1 score and combine the information from precision and recall. 
* The random forest model yields a more effective performance than a decision tree model. 

**What summary would you provide to stakeholders?**
* The random forest model predicted satisfaction with more than 94.2% accuracy. The precision is over 95% and the recall is approximately 94.5%. 
* The random forest model outperformed the tuned decision tree with the best hyperparameters in most of the four scores. This indicates that the random forest model may perform better.
* Because stakeholders were interested in learning about the factors that are most important to customer satisfaction, this would be shared based on the tuned random forest. 
* In addition, you would provide details about the precision, recall, accuracy, and F1 scores to support your findings. 

### References

[What is the Difference Between Test and Validation Datasets?,  Jason Brownlee](https://machinelearningmastery.com/difference-test-validation-datasets/)

[Decision Trees and Random Forests Neil Liberman](https://towardsdatascience.com/decision-trees-and-random-forests-df0c3123f991)

**Congratulations!** You've completed this lab. However, you may not notice a green check mark next to this item on Coursera's platform. Please continue your progress regardless of the check mark. Just click on the "save" icon at the top of this notebook to ensure your work has been logged